# U-Net Training Notebook

In [1]:
!pip install wandb
!rm -r sample_data
!git clone https://github.com/PhilipMathieu/unet-orthoimagery.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.6 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=592237ac26f9bc14b8de7da234f9ff5c453790af6d884c255df2aa89314f2940
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools
Cloning into 'unet-orthoimagery'...
remote: Enumerating objects: 104885, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 104885 (delta 23), reused 35 (delta 11), pack-reused 104825


In [5]:
import sys
sys.path.append('./unet-orthoimagery/')
sys.path.append('./unet-orthoimagery/src/')

[WandB Quickstart](https://docs.wandb.ai/quickstart#:~:text=Provide%20your%20API%20key%20when%20prompted.) (including link to your API key, if logged in)

In [3]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
import logging
import torch
from src.train import train_model
from src.unet.unet_model import UNet

In [7]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device: {device}')

In [8]:
model = UNet(n_channels=4, n_classes=1, bilinear=False)
model = model.to(memory_format=torch.channels_last)

In [9]:
logging.info(f'Network:\n'
                 f'\t{model.n_channels} input channels\n'
                 f'\t{model.n_classes} output channels (classes)\n'
                 f'\t{"Bilinear" if model.bilinear else "transposed conv"} upscaling')


model.to(device=device);

In [13]:
data_dir = "unet-orthoimagery/data/Image_Chips_128_overlap_balanced_dem/"

In [ ]:
try:
    train_model(
        model=model,
        epochs=20,
        device=device,
        data_dir=data_dir
    )
except torch.cuda.OutOfMemoryError: # Giving me syntax error saying '"OutOfMemoryError" is not a valid exception class.'
    logging.error('Detected OutOfMemoryError! '
                    'Enabling checkpointing to reduce memory usage, but this slows down training. '
                    'Consider enabling AMP (--amp) for fast and memory efficient training')
    torch.cuda.empty_cache()
    model.use_checkpointing()
    train_model(
        model=model,
        epochs=20,
        device=device,
        data_dir=data_dir
    )

100%|██████████| 640/640 [00:00<00:00, 721.22it/s]
wandb: Currently logged in as: philipmathieu (unet-ortho). Use `wandb login --relogin` to force relogin


Epoch 1/20:   0%|          | 0/427 [00:00<?, ?img/s]

Validation round:   0%|          | 0/13 [00:00<?, ?batch/s]